In [57]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [6]:
X = torch.arange(0, 10, 1).reshape(10, 1).to(torch.float32)
X

tensor([[0.],
        [1.],
        [2.],
        [3.],
        [4.],
        [5.],
        [6.],
        [7.],
        [8.],
        [9.]])

In [73]:
class SeqAE(nn.Module):
	def __init__(self, window_size=3, hidden_size=32):
		super().__init__()
		self.window_size = window_size
		self.hidden_size = hidden_size
		self.encoder = nn.Sequential(
			nn.Linear(window_size, hidden_size),
			nn.ReLU(),
			nn.Linear(hidden_size, hidden_size),
		)
		self.decoder = nn.Sequential(
			nn.Linear(hidden_size, hidden_size),
			nn.ReLU(),
			nn.Linear(hidden_size, window_size),
		)

	def forward(self, X, cur_embed=None):
		if cur_embed is not None:
			return None, self.decoder(cur_embed)
		embed = self.encoder(X)
		return embed, self.decoder(embed)

model = SeqAE()

In [77]:
_X = X[:3].T
model = SeqAE()
optim = torch.optim.Adam(model.parameters(), lr=0.01)
for i in range(10_000):
	avg_embedding = torch.zeros(1, model.hidden_size, requires_grad=True)
	for j in range(0, 10-3):
		_X = X[j:j+3].T
		embed = model.encoder(_X)
		avg_embedding += embed
		avg_embedding /= (10-3)
		output = model.decoder(avg_embedding)
		loss = F.mse_loss(output, _X)

		optim.zero_grad()
		loss.backward(retain_graph=True)
		optim.step()
		print(loss.item())

RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.

In [72]:
model(X[0:3].T)

(tensor([[-1.1044e-01,  7.6778e-02, -7.5444e-01, -4.7193e-02, -3.5585e-02,
          -1.0240e-02,  1.2201e-01,  9.2389e-02, -3.8819e-01,  2.3825e-04,
           2.7074e+00,  7.9815e-02, -5.2191e-01,  4.7439e-02,  3.9992e-02,
           1.1675e-02,  8.1547e-03,  7.9263e-02, -6.5563e-02,  3.1444e-02,
          -6.2912e-03,  3.5412e-01,  8.7691e-02, -8.8728e-02, -4.3509e-01,
           7.6206e-02, -3.3255e-02, -2.9341e-01,  5.8198e-02, -8.0172e-02,
          -1.0646e-02, -1.2777e-01]], grad_fn=<AddmmBackward0>),
 tensor([[0.0167, 1.0167, 2.0167]], grad_fn=<AddmmBackward0>))